In [ ]:
pip install flipside

In [ ]:
from flipside import Flipside
flipside = Flipside("d473fc3c-38f6-4208-9ed4-8a954cb75d6d", "https://api-v2.flipsidecrypto.xyz") #you may place your own API keys in the first section

In [ ]:
sql = """
SELECT
  date_trunc('hour', block_timestamp) as hour,
  count(distinct tx_hash) as tx_count
FROM ethereum.core.fact_transactions
WHERE block_timestamp >= GETDATE() - interval'7 days'
GROUP BY 1
"""

# Run the query against Flipside's query engine and await the results
query_result_set = flipside.query(sql)

### Number of type of checks:

In [ ]:
check_type_q = """
WITH txns as
(select  distinct tx_hash, transaction_fee as tx_fee
 from near.core.fact_transactions b
where (tx_receiver = 'v1.nadabot.near'
  or   tx_signer = 'v1.nadabot.near'
  and tx_succeeded = TRUE))
,
qmain as (
select  block_timestamp,
        ARGS,
        b.tx_hash,
        signer_id,
        ACTION_NAME,
        receiver_id,
        method_name,
        --args,
        deposit  / 1e24 as deposit
 from near.core.fact_actions_events_function_call b, txns
where b.tx_hash = txns.tx_hash
),

main as (
SELECT
    block_timestamp,
    signer_id,
    receiver_id,
    method_name,
    deposit,
    SPLIT_PART(PARSE_JSON(ARGS):provider_id, ':', 1) AS contract_address,
    SPLIT_PART(PARSE_JSON(ARGS):provider_id, ':', 2) AS check_type
from qmain)


select count(DISTINCT check_type) as check_type_count
from main
where check_type in (select DISTINCT check_type
                     from main where method_name = 'add_stamp') and
      check_type != 'has_complete_profile_check'


"""
query_result_set = flipside.query(check_type_q)
query_result_set

In [ ]:
import pandas as pd

pd.DataFrame(query_result_set.records,columns = query_result_set.columns)

### Average checks per user

In [ ]:
avg_check_per_user_q = """
WITH txns as
(select  distinct tx_hash, transaction_fee as tx_fee
 from near.core.fact_transactions b
where (tx_receiver = 'v1.nadabot.near'
  or   tx_signer = 'v1.nadabot.near'
  and tx_succeeded = TRUE))
,
qmain as (
select  block_timestamp,
        ARGS,
        b.tx_hash,
        signer_id,
        ACTION_NAME,
        receiver_id,
        method_name,
        --args,
        deposit  / 1e24 as deposit
 from near.core.fact_actions_events_function_call b, txns
where b.tx_hash = txns.tx_hash
),

main as (
SELECT
    block_timestamp,
    signer_id,
    receiver_id,
    method_name,
    deposit,
    SPLIT_PART(PARSE_JSON(ARGS):provider_id, ':', 1) AS contract_address,
    SPLIT_PART(PARSE_JSON(ARGS):provider_id, ':', 2) AS check_type
from qmain),

check_type_table as (
select distinct check_type
from main
where check_type != 'has_complete_profile_check'),

avg_count as (
select signer_id, count(check_type) as checks_count
from main
where check_type in (select check_type from check_type_table) and method_name = 'verify_stamp_callback'
group by signer_id
)

select avg(checks_count) from avg_count

"""
query_result_set = flipside.query(avg_check_per_user_q)

In [ ]:
pd.DataFrame(query_result_set.records,columns = query_result_set.columns)

### weight count

In [ ]:
weight_count_q = """
WITH txns as (
  select
    distinct tx_hash,
    transaction_fee as tx_fee
  from
    near.core.fact_transactions b
  where
    (
      tx_receiver = 'v1.nadabot.near'
      or tx_signer = 'v1.nadabot.near'
      and tx_succeeded = TRUE
    )
),
qmain as (
  select
    block_timestamp,
    ARGS,
    b.tx_hash,
    signer_id,
    ACTION_NAME,
    receiver_id,
    method_name,
    --args,
    deposit / 1e24 as deposit
  from
    near.core.fact_actions_events_function_call b,
    txns
  where
    b.tx_hash = txns.tx_hash
),

main as (
  SELECT
    block_timestamp,
    signer_id,
    receiver_id,
    method_name,
    deposit,
    SPLIT_PART(PARSE_JSON(ARGS) :provider_id, ':', 1) AS contract_address,
    SPLIT_PART(PARSE_JSON(ARGS) :provider_id, ':', 2) AS check_type,
    COALESCE(TRY_PARSE_JSON(ARGS):provider.default_weight, 0)::INT AS weight
  from
    qmain
  where receiver_id = 'v1.nadabot.near'
)

select signer_id,sum(weight) as total_weight
from main
where method_name = 'verify_stamp_callback'
group by signer_id
order by total_weight desc

-- provider and users

"""
query_result_set = flipside.query(weight_count_q)
query_result_set

### Weight threshold

In [ ]:
weight_threshold_q = """
WITH txns as (
  select
    distinct tx_hash,
    transaction_fee as tx_fee
  from
    near.core.fact_transactions b
  where
    (
      tx_receiver = 'v1.nadabot.near'
      or tx_signer = 'v1.nadabot.near'
      and tx_succeeded = TRUE
    )
),
qmain as (
  select
    block_timestamp,
    ARGS,
    b.tx_hash,
    signer_id,
    ACTION_NAME,
    receiver_id,
    method_name,
    --args,
    deposit / 1e24 as deposit
  from
    near.core.fact_actions_events_function_call b,
    txns
  where
    b.tx_hash = txns.tx_hash
),

main as (
  SELECT
    block_timestamp,
    signer_id,
    receiver_id,
    method_name,
    deposit,
    SPLIT_PART(PARSE_JSON(ARGS) :provider_id, ':', 1) AS contract_address,
    SPLIT_PART(PARSE_JSON(ARGS) :provider_id, ':', 2) AS check_type,
    COALESCE(TRY_PARSE_JSON(ARGS):provider.default_weight, 0)::INT AS weight
  from
    qmain
  where receiver_id = 'v1.nadabot.near'
),

temp as (
  select signer_id,sum(weight) as total_weight
  from main
  where method_name = 'verify_stamp_callback'
  group by signer_id
  order by total_weight desc
),

threshold as (
select TRY_PARSE_JSON(ARGS):default_human_threshold::INT as thresh
from qmain
where thresh is not null

)

select *
from ( select count(*) as above
       from temp
       where total_weight >= (select thresh from threshold)) a,
     (select avg(total_weight) as average_weight
       from temp) b, threshold c
"""
query_result_set = flipside.query(weight_threshold_q)
query_result_set

### user check count

In [ ]:
user_check_count_q = """
WITH txns as
(select  distinct tx_hash, transaction_fee as tx_fee
 from near.core.fact_transactions b
where (tx_receiver = 'v1.nadabot.near'
  or   tx_signer = 'v1.nadabot.near'
  and tx_succeeded = TRUE))
,
qmain as (
select  block_timestamp,
        ARGS,
        b.tx_hash,
        signer_id,
        ACTION_NAME,
        receiver_id,
        method_name,
        --args,
        deposit  / 1e24 as deposit
 from near.core.fact_actions_events_function_call b, txns
where b.tx_hash = txns.tx_hash
),

main as (
SELECT
    block_timestamp,
    signer_id,
    receiver_id,
    method_name,
    SPLIT_PART(PARSE_JSON(ARGS):provider_id, ':', 1) AS contract_address,
    SPLIT_PART(PARSE_JSON(ARGS):provider_id, ':', 2) AS check_type
from qmain),

check_type_table as (
select distinct check_type
from main
where check_type != 'has_complete_profile_check')

select signer_id, count(method_name) as checks_count
from main
where check_type in (select check_type from check_type_table) and method_name = 'add_stamp'
group by signer_id
order by checks_count desc

"""
query_result_set = flipside.query(user_check_count_q)
query_result_set

### deployed contracts

In [ ]:
deployed_contracts_q = """
WITH txns as
(select  distinct tx_hash, transaction_fee as tx_fee
 from near.core.fact_transactions b
where (tx_receiver = 'creatives.v1.potfactory.potlock.near'
  or   tx_signer = 'creatives.v1.potfactory.potlock.near')
  and tx_succeeded = TRUE),

qmain as (
select  block_timestamp,
        b.tx_hash,
        signer_id,
        receiver_id,
        method_name,
        --args,
        deposit  / 1e24 as deposit
 from near.core.fact_actions_events_function_call b, txns
where b.tx_hash = txns.tx_hash
)
-- is_approved, apply/apply_for_assignment/apply_for_program
-- number of applications
select count(*)
from qmain
where method_name in ('is_approved','apply','apply_for_assignment','apply_for_program')

"""
query_result_set = flipside.query(deployed_contracts_q)
query_result_set

### Not human Table

In [ ]:
not_human_table_q = """
WITH txns as (
  select
    distinct tx_hash,
    transaction_fee as tx_fee
  from
    near.core.fact_transactions b
  where
    (
      tx_receiver = 'v1.nadabot.near'
      or tx_signer = 'v1.nadabot.near'
      and tx_succeeded = TRUE
    )
),
qmain as (
  select
    block_timestamp,
    ARGS,
    b.tx_hash,
    signer_id,
    ACTION_NAME,
    receiver_id,
    method_name,
    --args,
    deposit / 1e24 as deposit
  from
    near.core.fact_actions_events_function_call b,
    txns
  where
    b.tx_hash = txns.tx_hash
),

main as (
  SELECT
    block_timestamp,
    signer_id,
    receiver_id,
    method_name,
    deposit,
    SPLIT_PART(PARSE_JSON(ARGS) :provider_id, ':', 1) AS contract_address,
    SPLIT_PART(PARSE_JSON(ARGS) :provider_id, ':', 2) AS check_type,
    COALESCE(TRY_PARSE_JSON(ARGS):provider.default_weight, 0)::INT AS weight
  from
    qmain
  where receiver_id = 'v1.nadabot.near'
),

threshold as (
select TRY_PARSE_JSON(ARGS):default_human_threshold::INT as thresh
from qmain
where thresh is not null ),

not_human_list as (
select signer_id,sum(weight) as total_weight
from main
where method_name = 'verify_stamp_callback'
group by signer_id
having total_weight < (select thresh from threshold)
order by total_weight desc)

select m.signer_id,CONCAT(m.contract_address,':',m.check_type) as contactAD_checktype, m.weight
from main m
where m.signer_id in (select signer_id from not_human_list) and method_name = 'verify_stamp_callback'
order by m.signer_id

"""
query_result_set = flipside.query(not_human_table_q)
query_result_set

### If verified human make donation

In [ ]:
sql = """
WITH txns as (
  select
    distinct tx_hash,
    transaction_fee as tx_fee
  from
    near.core.fact_transactions b
  where
    (
      tx_receiver = 'v1.nadabot.near'
      or tx_signer = 'v1.nadabot.near'
      and tx_succeeded = TRUE
    )
),
qmain as (
  select
    block_timestamp,
    ARGS,
    b.tx_hash,
    signer_id,
    ACTION_NAME,
    receiver_id,
    method_name,
    --args,
    deposit / 1e24 as deposit
  from
    near.core.fact_actions_events_function_call b,
    txns
  where
    b.tx_hash = txns.tx_hash
),

main as (
  SELECT
    block_timestamp,
    signer_id,
    receiver_id,
    method_name,
    deposit,
    SPLIT_PART(PARSE_JSON(ARGS) :provider_id, ':', 1) AS contract_address,
    SPLIT_PART(PARSE_JSON(ARGS) :provider_id, ':', 2) AS check_type,
    COALESCE(TRY_PARSE_JSON(ARGS):provider.default_weight, 0)::INT AS weight
  from
    qmain
  where receiver_id = 'v1.nadabot.near'
),

threshold as (
select TRY_PARSE_JSON(ARGS):default_human_threshold::INT as thresh
from qmain
where thresh is not null ),

human_list as (
select signer_id,sum(weight) as total_weight
from main
where method_name = 'verify_stamp_callback'
group by signer_id
having total_weight >= (select thresh from threshold)
order by total_weight desc),

txns_d as
(select  distinct a.tx_hash, transaction_fee as tx_fee
 from near.core.fact_actions_events_function_call a, near.core.fact_transactions b
where receiver_id = 'donate.potlock.near'
  and method_name = 'donate'
  and a.tx_hash = b.tx_hash
  and tx_succeeded = TRUE)
,
qmain_d as (
select  block_timestamp,
        signer_id,
        receiver_id,
        try_parse_json(b.action_data):"deposit"::float / 1e24 as deposit,
        txns_d.tx_fee::float / 1e24 as tx_fee
 from near.core.fact_actions_events b, txns_d
where b.tx_hash = txns_d.tx_hash
  and b.action_name = 'Transfer'
  and b.receiver_id <> b.signer_id
  and receiver_id <> 'impact.sputnik-dao.near'
)

select *
from qmain_d
where qmain_d.signer_id in (select distinct signer_id from human_list)

-- select m.signer_id,CONCAT(m.contract_address,':',m.check_type) as contactAD_checktype, m.weight
-- from main m
-- where m.signer_id in (select signer_id from human_list) and method_name = 'verify_stamp_callback'
-- order by m.signer_id
"""
query_result_set = flipside.query(sql)

In [ ]:
sql = """
WITH txns as
(select  distinct tx_hash, transaction_fee as tx_fee
 from near.core.fact_transactions b
where (tx_receiver = 'v1.nadabot.near'
  or   tx_signer = 'v1.nadabot.near'
  and tx_succeeded = TRUE))
,
qmain as (
select  block_timestamp,
        ARGS,
        b.tx_hash,
        signer_id,
        ACTION_NAME,
        receiver_id,
        method_name,
        --args,
        deposit  / 1e24 as deposit
 from near.core.fact_actions_events_function_call b, txns
where b.tx_hash = txns.tx_hash
),

main as (
SELECT
    block_timestamp,
    signer_id,
    receiver_id,
    method_name,
    deposit,
    SPLIT_PART(PARSE_JSON(ARGS):provider_id, ':', 1) AS contract_address,
    SPLIT_PART(PARSE_JSON(ARGS):provider_id, ':', 2) AS check_type
from qmain),

second as (select contract_address,check_type,count(*) as user_count
from main
where method_name = 'verify_stamp_callback'
group by check_type,contract_address
order by user_count desc)

select *
from second
where check_type in (select DISTINCT check_type from main where method_name = 'add_stamp')
order by user_count DESC
"""
query_result_set = flipside.query(sql)

In [ ]:
query_result_set